In [1]:
import nltk
import random
import os
import json
from nltk.stem import  PorterStemmer
import random
import pickle

In [2]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset_new'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,each_file) for sentence in data_temp]
    return dataset